# s3 range-get requests w/ AWS SDK

In [ ]:
"""
Try to manually execute some S3 range-get requests 
using lower-level Python code (e.g., Python AWS SDK — 
I think this is the boto library) and decompress and 
decode them into numpy arrays using the information in the 
Kerchunk reference files (byte order, dimensionality, etc.). 
This will help you understand the steps youll eventually need to 
do in C++, and help you make sure what youre doing is actually possible 
(e.g., there arent weird web issues that arent your fault) in a 
friendlier interactive environment.

"""

